# Sortie arrière-pays G5 1789 ; récupération des données pour avoir les congés par ports et par provinces associés

Récupérer les données du document decompte_sources_saisies

In [ ]:
import requests
import csv
from pprint import pprint

def get_online_csv(url):
  results = []
  with requests.Session() as s:
      download = s.get(url)
      decoded_content = download.content.decode('utf-8')
      reader = csv.DictReader(decoded_content.splitlines(), delimiter=',')
      for row in reader:
        results.append(dict(row))
  return results

sorties_g5 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRNAeIEFhB_RTm2xBgeuXl5oMtNIrGhWT6uCB2S9wEUblwDidRBwv9dp8D0S-YIPUyoASaG2p-NgfWD/pub?output=csv'

sorties_g5 = get_online_csv(sorties_g5)

pprint(sorties_g5[0])

print(type(sorties_g5))

Récupérer les provinces des pointcalls pour cibler l'arrière pays

In [ ]:
pip install pandas

In [ ]:
from csv import DictReader
import pandas as pd

with open('../../data/navigo_all_pointcalls.csv', newline='', encoding='utf8') as csvfile:
    pointcalls = pd.read_csv(csvfile, sep=',')

In [ ]:
pointcalls.info()
pointcalls

In [ ]:
data = pointcalls[['toponyme_fr', 'pointcall_province']]

data

Cibler l'arrière pays dans tous les pointcalls

In [ ]:
arriere_pays = ["Corse", "Provence", "Languedoc", "Roussillon"]

subset_data = data[data['pointcall_province'].isin(arriere_pays)]
subset_data

Supprimer les doublons 

In [ ]:
subset_data.drop_duplicates(keep = 'first', inplace=True)
subset_data

Jointure province des pointcalls avec le décompte des sorties G5 en 1789

In [ ]:
df_sorties_g5 = pd.DataFrame(sorties_g5, columns =['annee', 
                                                  'nb_conges_cr',
                                                  'nb_conges_inputdone',
                                                  'nb_conges_sante',
                                                  'nb_longcours_marseille',
                                                  'nb_petitcabotage',
                                                  'toponyme_standard_fr',
                                                  'uhgs_id'
                                                  ])

sortie_arriere_pays = pd.merge(df_sorties_g5, subset_data, left_on='toponyme_standard_fr', right_on='toponyme_fr')
sortie_arriere_pays

Filtrer par année 1789, et uniquement les noms des ports et leur province associées


In [ ]:
sortie_arriere_pays_1789 = sortie_arriere_pays[sortie_arriere_pays['annee'] == '1789']
sortie_arriere_pays_1789 = sortie_arriere_pays_1789[['nb_conges_cr', 'toponyme_fr', 'pointcall_province']]
sortie_arriere_pays_1789 

In [ ]:
from IPython.display import display

def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

In [ ]:
dict_sortie_arriere_pays_1789 = sortie_arriere_pays_1789.to_dict('records')

pprint(dict_sortie_arriere_pays_1789)

Nombre de congés en fonction du port

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Nombre de congés en fonction du port",
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "bar" #type de visualisation (ici rectangle)
    },
    "data": {
        "values": dict_sortie_arriere_pays_1789 #mettre la liste des données
    },
    "encoding": {
        "x": {
            "field": "nb_conges_cr",
            "type": "quantitative", #ou nominal 
            "title": "conges",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "field": "toponyme_fr",
            "type": "nominal",
            "title": "port",
            "sort": {"field" : "nb_conges_cr",
                     "order": "descending"}
        },
    }
})


Nombre de congés en fonction de la province

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Nombre de congés en fonction de la province",
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "bar" #type de visualisation (ici rectangle)
    },
    "data": {
        "values": dict_sortie_arriere_pays_1789 #mettre la liste des données
    },
    "encoding": {
        "x": {
            "field": "nb_conges_cr",
            "type": "quantitative", #ou nominal 
            "title": "conges",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "field": "pointcall_province",
            "type": "nominal",
            "title": "port",
            "sort": {"field" : "nb_conges_cr",
                     "order": "descending"}
        },
    }
})
